In [1]:
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from PIL import Image

In [2]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.losses import *
from tensorflow.keras import activations

In [3]:
from wide_resnet import WideResidualNetwork as wresnet

In [4]:
def load_cifar(filespath,labelspath):
    images_list = []
    df = pd.read_csv(labelspath)
    labels_arr = df.iloc[:,1].values
    coor_labels = []
    for filename in glob.glob(filespath+'/*.png'):
        fileNo = int(filename.split('/')[-1].split('.')[0])-1
        im=Image.open(filename)
        images_list.append(np.array(im))
        coor_labels.append(labels_arr[fileNo])
    return np.asarray(images_list), np.asarray(coor_labels)

In [5]:
X_dataset, y_dataset = load_cifar('./train','./train_labels.csv')

In [6]:
X_dataset = X_dataset.astype('float32')
X_dataset = (X_dataset-127.5)/127.5
X_dataset.shape

(47500, 32, 32, 3)

In [7]:
y_dataset = tf.keras.utils.to_categorical(y_dataset)
y_dataset.shape

(47500, 10)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset,
                                                    y_dataset,
                                                    test_size=0.2)

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_gen_train = ImageDataGenerator(rotation_range=20,width_shift_range=0.2,
                                    height_shift_range=0.2,horizontal_flip=True)
train_data_gen = image_gen_train.flow(X_train,y_train,batch_size=32)

In [11]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.losses import *
from tensorflow.keras import activations

In [24]:
model = wresnet()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-6),loss="categorical_crossentropy",metrics=['accuracy'])

In [25]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau()
model.fit_generator(train_data_gen,steps_per_epoch=X_train.shape[0]//32,validation_data=(X_test,y_test),
                   epochs=15,callbacks=[reduce_lr])

Epoch 1/15
1187/1187 [==============================] - 178s 150ms/step - loss: 0.3946 - accuracy: 0.8800 - val_loss: 0.0688 - val_accuracy: 0.9760
Epoch 2/15
1187/1187 [==============================] - 179s 151ms/step - loss: 0.3815 - accuracy: 0.8837 - val_loss: 0.0646 - val_accuracy: 0.9771
Epoch 3/15
1187/1187 [==============================] - 178s 150ms/step - loss: 0.3576 - accuracy: 0.8901 - val_loss: 0.0633 - val_accuracy: 0.9780
Epoch 4/15
1187/1187 [==============================] - 178s 150ms/step - loss: 0.3451 - accuracy: 0.8921 - val_loss: 0.0601 - val_accuracy: 0.9788
Epoch 5/15
1187/1187 [==============================] - 178s 150ms/step - loss: 0.3388 - accuracy: 0.8943 - val_loss: 0.0592 - val_accuracy: 0.9793
Epoch 6/15
1187/1187 [==============================] - 179s 150ms/step - loss: 0.3321 - accuracy: 0.8950 - val_loss: 0.0583 - val_accuracy: 0.9800
Epoch 7/15
1187/1187 [==============================] - 178s 150ms/step - loss: 0.3288 - accuracy: 0.8950 - val_

In [26]:
test_list = []
test_labels = []
for filename in glob.glob('./finaltest'+'/*.png'):
    idname = filename.split('/')[-1].split(')')[0].split('(')[-1]
    im=Image.open(filename)
    test_list.append(np.array(im))
    test_labels.append(idname)

In [27]:
test_list = np.asarray(test_list)
test_labels = np.asarray(test_labels)

In [28]:
test_list.astype('float32')
test_list = (test_list-127.5)/127.5
test_list.shape

(2500, 32, 32, 3)

In [40]:
test_result = model.predict(test_list)

In [41]:
test_result2 = test_result.argmax(axis=1)

In [42]:
submission = pd.DataFrame({ 'id': test_labels, 'class': test_result2 })
submission.to_csv("coic3_final_test_wres3.csv", index=False)